<a href="https://colab.research.google.com/github/mckang6113875/AP_Test_OPEN/blob/main/SVM_TensorFlow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
#6113875

In [11]:
! pip install tensorflow_graphics

In [12]:
# Copyright 2020 The TensorFlow Authors
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#    https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
"""Quadratic radial lens distortion and un-distortion functions.

Given a vector in homogeneous coordinates, `(x/z, y/z, 1)`, we define
`r^2 = (x/z)^2 + (y/z)^2`. We use the simplest form of distortion function,
`f(r) = 1 + k * r^2`. The distorted vector is given by
`(f(r) * x/z, f(r) * y/z, 1)`.

To apply the undistortion, we need the inverse of f(r), g = f^{-1}. In this
library we use the approximate formula for the undistortion function given here
https://www.ncbi.nlm.nih.gov/pmc/articles/PMC4934233/, and refine the solution
using Newton-Raphson iterations (https://en.wikipedia.org/wiki/Newtons_method).

Restricting the distortion function to quadratic form allows to easily detect
the cases where `r` goes beyond the monotonically-increasing range of `f` (which
we refer to as overflow).
"""

from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

from typing import Tuple
from six.moves import range
import tensorflow as tf

from tensorflow_graphics.util import asserts
from tensorflow_graphics.util import export_api
from tensorflow_graphics.util import shape
from tensorflow_graphics.util import type_alias


def distortion_factor(
    squared_radius: type_alias.TensorLike,
    distortion_coefficient: type_alias.TensorLike,
    name: str = "quadratic_radial_distortion_distortion_factor"
) -> Tuple[tf.Tensor, tf.Tensor]:
  """Calculates a quadratic distortion factor given squared radii.

  Given a vector describing a location in camera space in homogeneous
  coordinates, `(x/z, y/z, 1)`, squared_radius is `r^2 = (x/z)^2 + (y/z)^2`.
  distortion_factor multiplies `x/z` and `y/z` to obtain the distorted
  coordinates. In this function, `distortion_factor` is given by
  `1.0 + distortion_coefficient * squared_radius`.

  Note:
    In the following, A1 to An are optional batch dimensions, which must be
    broadcast compatible.

  Args:
    squared_radius: A tensor of shape `[A1, ..., An, H, W]`, containing the
      radii of the image pixels computed as `(x/z)^2 + (y/z)^2`. We use squared
      radius rather than the radius itself to avoid an unnecessary `sqrt`, which
      may introduce gradient singularities. The non-negativity of squared radius
      is only enforced in debug mode.
    distortion_coefficient: A `scalar` or a tensor of shape `[A1, ..., An]`,
      which contains the distortion coefficients of each image.
    name: A name for this op. Defaults to
      "quadratic_radial_distortion_distortion_factor".

  Returns:
    distortion_factor: A tensor of shape `[A1, ..., An, H, W]`, the correction
      factor that should multiply the projective coordinates `(x/z)` and `(y/z)`
      to apply the distortion.
    overflow_mask: A boolean tensor of shape `[A1, ..., An, H, W]`, `True` where
      `squared_radius` is beyond the range where the distortion function is
      monotonically increasing. Wherever `overflow_mask` is True,
      `distortion_factor`'s value is meaningless.
  """
  with tf.name_scope(name,):
    squared_radius = tf.convert_to_tensor(value=squared_radius)
    distortion_coefficient = tf.convert_to_tensor(value=distortion_coefficient)

    if distortion_coefficient.shape.ndims == 0:
      distortion_coefficient = tf.expand_dims(distortion_coefficient, axis=0)
    shape.check_static(
        tensor=squared_radius,
        tensor_name="squared_radius",
        has_rank_greater_than=1)
    shape.compare_batch_dimensions(
        tensors=(squared_radius, distortion_coefficient),
        tensor_names=("squared_radius", "distortion_coefficient"),
        last_axes=(-3, -1),
        broadcast_compatible=True)
    squared_radius = asserts.assert_all_above(
        squared_radius, 0.0, open_bound=False)
    distortion_coefficient = tf.expand_dims(distortion_coefficient, axis=-1)
    distortion_coefficient = tf.expand_dims(distortion_coefficient, axis=-1)
    distortion_coefficient_times_squared_radius = (
        distortion_coefficient * squared_radius)
    distortion_factor_ = 1.0 + distortion_coefficient_times_squared_radius
    # This condition needs to hold for the distortion to be monotomnically
    # increasing, as can be derived by differentiating it.
    overflow_mask = tf.less(
        1.0 + 3.0 * distortion_coefficient_times_squared_radius, 0.0)
    return distortion_factor_, overflow_mask


def undistortion_factor(
    distorted_squared_radius: type_alias.TensorLike,
    distortion_coefficient: type_alias.TensorLike,
    num_iterations: int = 5,
    name: str = "quadratic_radial_distortion_undistortion_factor"
) -> Tuple[tf.Tensor, tf.Tensor]:
  """Calculates the inverse quadratic distortion function given squared radii.

  Given a vector describing a location in camera space in homogeneous
  coordinates `(x/z, y/z, 1)`, after distortion has been applied, these become
  `(x'/z, y'/z, 1)`. `distorted_squared_radius` is `(x'/z)^2 + (y'/z)^2`.
  `undistortion_factor` multiplies `x'/z` and `y'/z` to obtain the undistorted
  projective coordinates `x/z` and `y/z`.
  The undustortion factor in this function is derived from a quadratic.
  distortion function, where the distortion factor equals
  `1.0 + distortion_coefficient * squared_radius`.

  Note:
    In the following, A1 to An are optional batch dimensions, which must be
    broadcast compatible.

  Args:
    distorted_squared_radius: A tensor of shape `[A1, ..., An, H, W]` containing
      the value of  projective coordinates `(x/z)^2 + (y/z)^2`. For each pixel
      it contains the squared distance of that pixel to the center of the image
      plane. We use `distorted_squared_radius` rather than the distorted radius
      itself to avoid an unnecessary `sqrt`, which may introduce gradient
      singularities. The non-negativity of `distorted_squared_radius` is only
      enforced in debug mode.
    distortion_coefficient: A `scalar` or a tensor of shape `[A1, ..., An]`,
      which contains the distortion coefficients of each image.
    num_iterations: Number of Newton-Raphson iterations to calculate the inverse
      distortion function. Defaults to 5, which is on the high-accuracy side.
    name: A name for this op. Defaults to
      "quadratic_radial_distortion_undistortion_factor".

  Returns:
    undistortion: A tensor of shape `[A1, ..., An, H, W]` containing the
      correction factor that should multiply the distorted projective
      coordinates `(x'/z)` and `(y'/z)` to obtain the undistorted ones.
    overflow_mask: A `bool` tensor of shape `[A1, ..., An, H, W]`, `True` where
      `distorted_squared_radius` is beyond the range where the distortion
      function is monotonically increasing. Wherever `overflow_mask` is `True`,
      `undistortion_factor`'s value is meaningless.

  """
  with tf.name_scope(name):
    distorted_squared_radius = tf.convert_to_tensor(
        value=distorted_squared_radius)
    distortion_coefficient = tf.convert_to_tensor(value=distortion_coefficient)

    if distortion_coefficient.shape.ndims == 0:
      distortion_coefficient = tf.expand_dims(distortion_coefficient, axis=0)
    shape.check_static(
        tensor=distorted_squared_radius,
        tensor_name="distorted_squared_radius",
        has_rank_greater_than=1)
    shape.compare_batch_dimensions(
        tensors=(distorted_squared_radius, distortion_coefficient),
        tensor_names=("distorted_squared_radius", "distortion_coefficient"),
        last_axes=(-3, -1),
        broadcast_compatible=True)
    distorted_squared_radius = asserts.assert_all_above(
        distorted_squared_radius, 0.0, open_bound=False)
    distortion_coefficient = tf.expand_dims(distortion_coefficient, axis=-1)
    distortion_coefficient = tf.expand_dims(distortion_coefficient, axis=-1)
    # For a distortion function of r' = (1 + ar^2)r, with a negative a, the
    # maximum r until which r'(r) is monotonically increasing is r^2 = -1/(3a).
    # At that value, r'^2 = -4 / (27a). Therefore the overflow condition for r'
    # is ar'^2 +(4/27.0) < 0. For a positive a it never holds, as it should,
    # because then r' is monotonic in r everywhere and thus never overflows.
    distortion_coefficient_times_distorted_squared_radius = (
        distortion_coefficient * distorted_squared_radius)
    overflow_mask = tf.less(
        4.0 / 27.0 + distortion_coefficient_times_distorted_squared_radius, 0.0)

    # Newton-raphson iterations. The expression below is obtained from
    # algebrically simplifying the Newton-Raphson formula
    # (https://en.wikipedia.org/wiki/Newtons_method).
    # We initialize with the approximate formula for the undistortion function
    # given here https://www.ncbi.nlm.nih.gov/pmc/articles/PMC4934233/.
    undistortion = (1.0 - distortion_coefficient_times_distorted_squared_radius)
    for _ in range(num_iterations):
      two_thirds_undistortion = 2.0 * undistortion / 3.0
      undistortion = (1.0 - two_thirds_undistortion) / (
          1.0 + 3.0 * distortion_coefficient_times_distorted_squared_radius *
          undistortion * undistortion) + two_thirds_undistortion
    return undistortion, overflow_mask


# API contains all public functions and classes.
__all__ = export_api.get_functions_and_classes()

In [13]:
# Copyright 2020 The TensorFlow Authors
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#    https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
"""tensorflow_graphics module."""
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

# pylint: disable=g-statement-before-imports,g-import-not-at-top
try:
  import tensorflow as tf
except ImportError:
  print("Warning: TensorFlow is not installed when you install TensorFlow"
        " Graphics. To use TensorFlow Graphics, please install TensorFlow, by"
        " following instructions at https://tensorflow.org/install or by using"
        " pip install tensorflow_graphics[tf] or"
        " pip install tensorflow_graphics[tf-gpu].")
# pylint: enable=g-statement-before-imports,g-import-not-at-top

# pylint: disable=g-statement-before-imports,g-import-not-at-top,ungrouped-imports
from tensorflow_graphics.util.doc import _import_tfg_docs
if _import_tfg_docs():
  from tensorflow_graphics import datasets
  from tensorflow_graphics import geometry
  from tensorflow_graphics import image
  from tensorflow_graphics import math
  from tensorflow_graphics import nn
  from tensorflow_graphics import notebooks
  from tensorflow_graphics import projects
  from tensorflow_graphics import rendering
  from tensorflow_graphics import util

  # submodules of tensorflow_graphics
  __all__ = util.export_api.get_modules()

  # Remove modules projects, notebooks, util and version from API.
  __all__.remove("projects")
  __all__.remove("notebooks")
  __all__.remove("util")
# pylint: enable=g-statement-before-imports,g-import-not-at-top

__version__ = "HEAD"

In [14]:
def distort(images, d, name='distort'):
        def _repeat(x, n_repeats):
            with tf.variable_scope('_repeat'):
                rep = tf.transpose(
                tf.expand_dims(tf.ones(shape=tf.stack([n_repeats, ])), 1), [1, 0])
                rep = tf.cast(rep, 'int32')
                x = tf.matmul(tf.reshape(x, (-1, 1)), rep)
                return tf.reshape(x, [-1])

        def _interpolate(im, x, y, out_size):
            with tf.variable_scope('_interpolate'):
                # constants
                num_batch = tf.shape(im)[0]
                height = tf.shape(im)[1]
                width = tf.shape(im)[2]
                channels = tf.shape(im)[3]

                x = tf.cast(x, 'float32')
                y = tf.cast(y, 'float32')
                height_f = tf.cast(height, 'float32')
                width_f = tf.cast(width, 'float32')
                out_height = out_size[0]
                out_width = out_size[1]
                zero = tf.zeros([], dtype='int32')
                max_y = tf.cast(tf.shape(im)[1] - 1, 'int32')
                max_x = tf.cast(tf.shape(im)[2] - 1, 'int32')

                # scale indices from [-1, 1] to [0, width/height]
                x = (x + 1.0)*(width_f) / 2.0
                y = (y + 1.0)*(height_f) / 2.0

                # do sampling
                x0 = tf.cast(tf.floor(x), 'int32')
                x1 = x0 + 1
                y0 = tf.cast(tf.floor(y), 'int32')
                y1 = y0 + 1

                x0 = tf.clip_by_value(x0, zero, max_x)
                x1 = tf.clip_by_value(x1, zero, max_x)
                y0 = tf.clip_by_value(y0, zero, max_y)
                y1 = tf.clip_by_value(y1, zero, max_y)
                dim2 = width
                dim1 = width*height
                base = _repeat(tf.range(num_batch)*dim1, out_height*out_width)
                base_y0 = base + y0*dim2
                base_y1 = base + y1*dim2
                idx_a = base_y0 + x0
                idx_b = base_y1 + x0
                idx_c = base_y0 + x1
                idx_d = base_y1 + x1

                # use indices to lookup pixels in the flat image and restore
                # channels dim
                im_flat = tf.reshape(im, tf.stack([-1, channels]))
                im_flat = tf.cast(im_flat, 'float32')
                Ia = tf.gather(im_flat, idx_a)
                Ib = tf.gather(im_flat, idx_b)
                Ic = tf.gather(im_flat, idx_c)
                Id = tf.gather(im_flat, idx_d)

                # and finally calculate interpolated values
                x0_f = tf.cast(x0, 'float32')
                x1_f = tf.cast(x1, 'float32')
                y0_f = tf.cast(y0, 'float32')
                y1_f = tf.cast(y1, 'float32')
                wa = tf.expand_dims(((x1_f-x) * (y1_f-y)), 1)
                wb = tf.expand_dims(((x1_f-x) * (y-y0_f)), 1)
                wc = tf.expand_dims(((x-x0_f) * (y1_f-y)), 1)
                wd = tf.expand_dims(((x-x0_f) * (y-y0_f)), 1)
                output = tf.add_n([wa*Ia, wb*Ib, wc*Ic, wd*Id])
                return output

            def _transform(images, d, out_size):
                with tf.variable_scope('_transform'): 

                    shape = tf.shape(images)
                    num_batch = tf.shape(images)[0]
                    num_channels = images.get_shape()[3]

                    out_width = out_size[1]
                    out_height = out_size[0]
                    cx = fx = fy = tf.to_float(out_width) / 2
                    cy = tf.to_float(out_height) / 2
                    x = tf.linspace(-1., 1., out_width)
                    y = tf.linspace(-1., 1., out_height)
                    x, y = tf.meshgrid(x, y)
                    x = tf.tile(tf.reshape(x, [1, -1, 1]), [num_batch,1,1])
                    y = tf.tile(tf.reshape(y, [1, -1, 1]), [num_batch,1,1])

                    a = x 
                    b = y 

                    r2 = tf.square(a) + tf.square(b)

                    r = tf.sqrt(r2)
                    r = tf.Print(r, [tf.reduce_min(r), tf.reduce_max(r)], "R min/max: ")
                    theta = tf.atan(r)
                    theta_d = theta*(1.0 + tf.reduce_sum(tf.reshape(d,
                          [1,1,4]) * tf.concat([tf.square(theta),
                          tf.pow(theta, 4), tf.pow(theta, 6), tf.pow(theta, 
                          8)], axis=-1),
                          axis=-1, keepdims=True))
                    tdr = theta_d / r
                    xd = a * tdr
                    yd = b * tdr


                    xd = tf.reshape(xd, [-1])
                    yd = tf.reshape(yd, [-1])

                    input_transformed = _interpolate(
                            images, xd, yd,
                            out_size)
                    output = tf.reshape(input_transformed, 
                             tf.stack([num_batch, out_height, out_width, 
                             num_channels]))
                    return output
            with tf.variable_scope(name):
                output = _transform(images, d, tf.shape(images)[1:3])
                return output

In [15]:
# Copyright 2020 The TensorFlow Authors
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#    https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
"""Camera feature."""

from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import numpy as np
import tensorflow as tf
from tensorflow_datasets import features

from tensorflow_graphics.datasets.features import pose_feature


class Camera(features.FeaturesDict):
  """`FeatureConnector` for camera calibration (extrinsic and intrinsic).

  During `_generate_examples`, the feature connector accepts as input:

    * `parameter_dict:` A dictionary containing the extrinsic and instrinsic
    parameters of the camera as:
      - 'pose': Dictionary containing
          * Either 3x3 rotation matrix and translation vector:
            {
              'R': A `float32` tensor with shape `[3, 3]` denoting the
                   3D rotation matrix.
              't': A `float32` tensor with shape `[3,]` denoting the
                   translation vector.
            }
      OR
         * look_at, position and up-vector:
            {
              'look_at': float32 vector of shape (3,).
              'position': float32 vector of shape (3,).
              'up': float32 vector of shape (3,).
            }
      - 'f': focal length of the camera in pixel (either single float32 value
      or tuple of float32 as (f_x, f_y).
      - 'optical_center': Optical center of the camera
      in pixel coordinates as tuple (c_x, c_y)
      Optional parameters:
      - 'skew': float32 denoting the skew of the camera axes.
      - 'aspect_ratio': float32 denoting the aspect_ratio,
      if single fixed focal length is provided.


  Output:
    A dictionary containing:

    * 'pose': A `tensorflow_graphics.datasets.features.Pose` FeatureConnector
    representing the 3D pose of the camera.
    * 'intrinsics': A `float32` tensor with shape `[3,3]` denoting the intrinsic
    matrix.

  Example:
    Default values for skew (s) and aspect_ratio(a) are 0 and 1, respectively.

    Full calibration matrix:
      K = [[ f_x,   s, c_x ],
           [   0, f_y, c_y ],
           [   0,   0,  1  ]]

    With same focal length:
      K = [[ f,  s, c_x ],
           [ 0, af, c_y ],
           [ 0,  0,  1  ]]
  """

  def __init__(self):
    super(Camera, self).__init__({
        'pose': pose_feature.Pose(),
        'intrinsics': features.Tensor(shape=(3, 3), dtype=tf.float32),
    })

  def encode_example(self, example_dict):
    """Convert the given parameters into a dict convertible to tf example."""
    REQUIRED_KEYS = ['pose', 'f', 'optical_center']  # pylint: disable=invalid-name
    if not all(key in example_dict for key in REQUIRED_KEYS):
      raise ValueError(f'Missing keys in provided dictionary! '
                       f'Expected {REQUIRED_KEYS}, '
                       f'but {example_dict.keys()} were given.')

    if not isinstance(example_dict['pose'], dict):
      raise ValueError('Pose needs to be a dictionary containing either '
                       'rotation and translation or look at, '
                       'up vector and position.')
    features_dict = {}
    pose_dict = example_dict['pose']
    if all(key in pose_dict for key in ['R', 't']):
      features_dict['pose'] = {
          'R': pose_dict['R'],
          't': pose_dict['t']
      }
    elif all(key in pose_dict for key in ['look_at', 'position', 'up']):
      rotation = self._create_rotation_from_look_at(pose_dict['look_at'],
                                                    pose_dict['position'],
                                                    pose_dict['up'])
      translation = (-rotation) @ pose_dict['position']

      features_dict['pose'] = {
          'R': rotation,
          't': translation
      }
    else:
      raise ValueError('Wrong keys for pose feature provided!')

    aspect_ratio = 1
    skew = 0
    if 'aspect_ratio' in example_dict.keys():
      if not isinstance(example_dict['f'], float):
        raise ValueError('If aspect ratio is provided, '
                         'f needs to be a single float.')
      aspect_ratio = example_dict['aspect_ratio']

    if 'skew' in example_dict.keys():
      skew = example_dict['skew']

    features_dict['intrinsics'] = self._create_calibration_matrix(
        example_dict['f'],
        example_dict['optical_center'],
        aspect_ratio,
        skew
    )

    return super(Camera, self).encode_example(features_dict)

  def _create_rotation_from_look_at(self, look_at, position, up):
    """Creates rotation matrix according to OpenGL gluLookAt convention.

    Args:
      look_at: A float32 3D vector of look_at direction.
      position: A float32 3D vector of camera position.
      up: A float32 3D up direction vector.

    Returns:
      A 3x3 float32 rotation matrix.

    (https://www.khronos.org/registry/OpenGL-Refpages/gl2.1/xhtml/gluLookAt.xml)
    """
    dir_vec = look_at - position
    dir_vec /= np.linalg.norm(dir_vec)
    side_vec = np.cross(dir_vec, up)
    side_vec /= np.linalg.norm(side_vec)
    up_vec = np.cross(side_vec, dir_vec)
    matrix = np.array([side_vec, up_vec, -dir_vec])

    return matrix

  def _create_calibration_matrix(self, f, optical_center, aspect_ratio=1,
                                 skew=0):
    """Constructs the 3x3 calibration matrix K.

    Args:
      f: Focal length of the camera. Either single float.32 value or tuple of
        float32 when different focal lengths for each axis are provided (fx, fy)
      optical_center: Tuple (c_x, c_y) containing the optical center
        of the camera in pixel coordinates.
      aspect_ratio: Optional parameter, if fixed focal length for both
        dimensions is used. Defaults to 1.
      skew: Optional parameter denoting the skew between the camera axes.

    Returns:
      float32 Tensor of shape [3,3] containing the upper triangular
      calibration matrix K.
    """
    if not isinstance(optical_center, tuple):
      raise ValueError('Optical center of camera needs '
                       'to be a tuple of (c_x, c_y).')

    if isinstance(f, tuple):
      f_x, f_y = f
    else:
      f_x = f
      f_y = aspect_ratio * f

    return np.asarray([[f_x, skew, optical_center[0]],
                       [0, f_y, optical_center[1]],
                       [0, 0, 1]
                       ], dtype=np.float32)

  @classmethod
  def from_json_content(cls, value) -> 'Camera':
    return cls()

  def to_json_content(self):
    return {}

https://github.com/tensorflow/graphics